Define the parameters from Wiki for testing :

In [ ]:
headings="""
name                | sh:tx:rop | mem | bw|bus|ocl|single|double|watts| pw                | amz"""
raw="""
GeForce GTX 750     | 512:32:16 | 1024| 80|128|1.2| 1044 | 32.6 |  55 |                   |B00IDG3NDY
GeForce GTX 750     | 512:32:16 | 2048| 80|128|1.2| 1044 | 32.6 |  55 |                   |B00J3ZNB04
GeForce GTX 760     |1152:96:32 | 2048|192|256|1.2| 2257 | 94   | 170 | geforce_gtx_760   |
GeForce GTX 760     |1152:96:32 | 4096|192|256|1.2| 2257 | 94   | 170 |                   |

GeForce GTX 960     |1024:64:32 | 2048|112|128|1.2| 2308 | 72.1 | 120 |                   |B00SC6HAS4
GeForce GTX 960     |1024:64:32 | 4096|112|128|1.2| 2308 | 72.1 | 120 |                   |B00UOYQ5LA

GeForce GTX 970     |1664:104:56| 3584|196|224|1.2| 3494 | 109  | 145 | geforce_gtx_970   |B00NVODXR4
GeForce GTX 980     |2048:128:64| 4096|224|256|1.2| 4612 | 144  | 165 | geforce_gtx_980   |B00NT9UT3M
GeForce GTX 980 Ti  |2816:176:96| 6144|336|384|1.2| 5632 | 176  | 250 |                   |B00YNEIAWY

GeForce GTX Titan X |3072:192:96|12288|336|384|1.2| 6144 | 192  | 250 | geforce_gtx_titan |B00UXTN5P0

"""
